In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [6]:
dataset=dataset.drop("User ID",axis=1)

In [7]:
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [8]:
dataset["Purchased"].value_counts()

0    257
1    143
Name: Purchased, dtype: int64

In [9]:
dataset.columns

Index(['Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [10]:
independent=dataset[['Age', 'EstimatedSalary','Gender_Male']]

In [11]:
independent

,Age,EstimatedSalary,Gender_Male
0,19,19000,1
1,35,20000,1
2,26,43000,0
3,27,57000,0
4,19,76000,1
...,...,...,...
395,46,41000,0
396,51,23000,1
397,50,20000,0
398,36,33000,1


In [12]:
dependent=dataset[['Purchased']]

In [13]:
dependent

,Purchased
0,0
1,0
2,0
3,0
4,0
...,...
395,1
396,1
397,1
398,0


In [14]:
independent.shape

(400, 3)

In [15]:
dependent.shape

(400, 1)

In [16]:
from sklearn.model_selection import  train_test_split
X_train,X_test,y_train,y_test=train_test_split(independent,dependent,test_size=1/3,random_state=0)

In [17]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.fit_transform(X_test)

In [19]:
from sklearn.ensemble import RandomForestClassifier

In [20]:
from sklearn.model_selection import GridSearchCV
param_grid = {'criterion':['gini','entropy'],'max_features':['auto','sqrt','log2'],
             'n_estimators':[10,100]}

grid=GridSearchCV(RandomForestClassifier(),param_grid,refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted')
grid.fit(X_train,y_train)

C:\Users\ramsm\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.


Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Done  20 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:    3.1s finished
C:\Users\ramsm\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\Users\ramsm\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:715: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
           

In [21]:
re = grid.cv_results_
grid_prediction = grid.predict(X_test)

In [22]:
from sklearn.metrics import confusion_matrix
cf=confusion_matrix(y_test,grid_prediction)

In [23]:
from sklearn.metrics import classification_report
clf_report=classification_report(y_test,grid_prediction)

In [24]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_prediction,average='weighted')
print("the f1_macro value for the best parameter {}:".format(grid.best_params_),f1_macro)

the f1_macro value for the best parameter {'criterion': 'entropy', 'max_features': 'sqrt', 'n_estimators': 100}: 0.9031880225910075


In [25]:
print ("the confusion Matrix:\n",cf)

the confusion Matrix:
 [[78  7]
 [ 6 43]]


In [26]:
print ("the report:\n",clf_report)

the report:
               precision    recall  f1-score   support

           0       0.93      0.92      0.92        85
           1       0.86      0.88      0.87        49

    accuracy                           0.90       134
   macro avg       0.89      0.90      0.90       134
weighted avg       0.90      0.90      0.90       134



In [27]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

0.9618247298919568

In [28]:
table=pd.DataFrame.from_dict(re)

In [29]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.014446,0.003443,0.005642,0.005134,gini,auto,10,"{'criterion': 'gini', 'max_features': 'auto', ...",0.841852,0.854397,0.884371,0.860069,0.017841,9
1,0.175036,0.008769,0.017629,0.002066,gini,auto,100,"{'criterion': 'gini', 'max_features': 'auto', ...",0.855026,0.877252,0.920734,0.884117,0.027304,5
2,0.044792,0.015080,0.002560,0.001820,gini,sqrt,10,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.807010,0.865289,0.885449,0.852240,0.033367,11
3,0.192520,0.013540,0.014725,0.001409,gini,sqrt,100,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.866667,0.876797,0.932275,0.891723,0.028812,2
4,0.028949,0.007732,0.005886,0.000365,gini,log2,10,"{'criterion': 'gini', 'max_features': 'log2', ...",0.831336,0.853686,0.883117,0.855860,0.021233,10
5,0.127591,0.002488,0.008624,0.002395,gini,log2,100,"{'criterion': 'gini', 'max_features': 'log2', ...",0.866667,0.865865,0.932275,0.888106,0.031058,4
6,0.024858,0.002516,0.003462,0.000959,entropy,auto,10,"{'criterion': 'entropy', 'max_features': 'auto...",0.853824,0.876797,0.920148,0.883366,0.027516,6
7,0.105552,0.005593,0.011003,0.000726,entropy,auto,100,"{'criterion': 'entropy', 'max_features': 'auto...",0.888889,0.876797,0.909091,0.891572,0.013274,3
8,0.011304,0.002418,0.001924,0.000121,entropy,sqrt,10,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.841852,0.842837,0.823629,0.836149,0.008813,12
9,0.099274,0.018895,0.005929,0.007011,entropy,sqrt,100,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.878187,0.888220,0.909091,0.891730,0.012878,1
